<a href="https://colab.research.google.com/github/VIJAY-VR-17/Tree-Species-Classification/blob/main/Tree_Species_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
